In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('titanic_processed.csv')

df.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
count,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000
mean,0.404494,2.240169,0.636236,29.642093,0.514045,0.432584,34.567251,0.182584,0.039326,0.778090
std,0.491139,0.836854,0.481420,14.492933,0.930692,0.854181,52.938648,0.386597,0.194506,0.415823
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,20.000000,0.000000,0.000000,8.050000,0.000000,0.000000,1.000000
50%,0.000000,2.000000,1.000000,28.000000,0.000000,0.000000,15.645850,0.000000,0.000000,1.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,1.000000,33.000000,0.000000,0.000000,1.000000
max,1.000000,3.000000,1.000000,80.000000,5.000000,6.000000,512.329200,1.000000,1.000000,1.000000


In [3]:
def summarize_classification(y_test, y_pred):
    
    acc = accuracy_score(y_test, y_pred, normalize=True)
    num_acc = accuracy_score(y_test, y_pred, normalize=False)

    prec = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    return {'accuracy': acc, 
            'precision': prec,
            'recall':recall, 
            'accuracy_count':num_acc}

In [4]:
FEATURES = list(df.columns[1:])
FEATURES

['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

In [5]:
def build_model(classifier_fn,                
                name_of_y_col, 
                names_of_x_cols, 
                dataset, 
                test_frac=0.2):
    
    X = dataset[names_of_x_cols]
    Y = dataset[name_of_y_col]

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_frac)
       
    model = classifier_fn(x_train, y_train)
    
    y_pred = model.predict(x_test)

    y_pred_train = model.predict(x_train)
    
    train_summary = summarize_classification(y_train, y_pred_train)
    test_summary = summarize_classification(y_test, y_pred)
    
    pred_results = pd.DataFrame({'y_test': y_test,
                                 'y_pred': y_pred})
    
    model_crosstab = pd.crosstab(pred_results.y_pred, pred_results.y_test)
    
    return {'training': train_summary, 
            'test': test_summary,
            'confusion_matrix': model_crosstab}

In [6]:
def compare_results():
    for key in result_dict:
        print('Classification: ', key)

        print()
        print('Training data')
        for score in result_dict[key]['training']:
            print(score, result_dict[key]['training'][score])

        print()
        print('Test data')
        for score in result_dict[key]['test']:
            print(score, result_dict[key]['test'][score])
       
        print()

In [7]:
def logistic_fn(x_train, y_train):
    
    model = LogisticRegression(solver='liblinear')
    model.fit(x_train, y_train)
    
    return model

In [8]:
result_dict = {}
result_dict['survived ~ logistic'] = build_model(logistic_fn,
                                              'Survived',
                                               FEATURES,
                                               df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.8014059753954306
precision 0.7853658536585366
recall 0.7
accuracy_count 456

Test data
accuracy 0.7902097902097902
precision 0.7916666666666666
recall 0.6551724137931034
accuracy_count 113



In [21]:
def linear_discriminant_fn(x_train, y_train, solver='svd'):
    
    model = LinearDiscriminantAnalysis(solver=solver)
    model.fit(x_train, y_train)
    
    return model

In [22]:
result_dict['survived ~ linear_discriminant_analysis'] = build_model(linear_discriminant_fn,
                                                                 'Survived',
                                                                  FEATURES,
                                                                  df)
compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.8014059753954306
precision 0.7864077669902912
recall 0.7012987012987013
accuracy_count 456

Test data
accuracy 0.8531468531468531
precision 0.95
recall 0.6666666666666666
accuracy_count 122

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7636363636363637
recall 0.717948717948718
accuracy_count 451

Test data
accuracy 0.8181818181818182
precision 0.78
recall 0.7222222222222222
accuracy_count 117



In [23]:
result_dict['survived ~ linear_discriminant_analysis'] = build_model(linear_discriminant_fn,
                                                                     'Survived',
                                                                      FEATURES[0:-1],
                                                                      df)
compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.8014059753954306
precision 0.7864077669902912
recall 0.7012987012987013
accuracy_count 456

Test data
accuracy 0.8531468531468531
precision 0.95
recall 0.6666666666666666
accuracy_count 122

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.8014059753954306
precision 0.7714285714285715
recall 0.7136563876651982
accuracy_count 456

Test data
accuracy 0.7762237762237763
precision 0.7735849056603774
recall 0.6721311475409836
accuracy_count 111



In [24]:
def quadratic_discriminant_fn(x_train, y_train):
    
    model = QuadraticDiscriminantAnalysis()
    model.fit(x_train, y_train)
    
    return model

In [26]:
result_dict['survived ~ quadratic_discriminant_analysis'] = build_model(quadratic_discriminant_fn,
                                                                        'Survived',
                                                                        FEATURES[0:-1],
                                                                        df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.8014059753954306
precision 0.7864077669902912
recall 0.7012987012987013
accuracy_count 456

Test data
accuracy 0.8531468531468531
precision 0.95
recall 0.6666666666666666
accuracy_count 122

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.8014059753954306
precision 0.7714285714285715
recall 0.7136563876651982
accuracy_count 456

Test data
accuracy 0.7762237762237763
precision 0.7735849056603774
recall 0.6721311475409836
accuracy_count 111

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.8
recall 0.721030042918455
accuracy_count 462

Test data
accuracy 0.7692307692307693
precision 0.7037037037037037
recall 0.6909090909090909
accuracy_count 110



In [27]:
def sgd_fn(x_train, y_train, max_iter=1000, tol=1e-3):
    
    model = SGDClassifier(max_iter=max_iter, tol=tol)
    model.fit(x_train, y_train)
     
    return model

In [29]:
result_dict['survived ~ sgd'] = build_model(sgd_fn,
                                           'Survived',
                                            FEATURES,
                                            df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.8014059753954306
precision 0.7864077669902912
recall 0.7012987012987013
accuracy_count 456

Test data
accuracy 0.8531468531468531
precision 0.95
recall 0.6666666666666666
accuracy_count 122

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.8014059753954306
precision 0.7714285714285715
recall 0.7136563876651982
accuracy_count 456

Test data
accuracy 0.7762237762237763
precision 0.7735849056603774
recall 0.6721311475409836
accuracy_count 111

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.8
recall 0.721030042918455
accuracy_count 462

Test data
accuracy 0.7692307692307693
precision 0.7037037037037037
recall 0.6909090909090909
accuracy_count 110

Classification:  survived ~ sgd

Training data
accuracy 0.7047451669595782
precision 0.8421052631578947
recall 0.2909090909090909
accuracy_count 401

Test data
accuracy 0.65034965034965

# LinearSVC

In [30]:
def linear_svc_fn(x_train, y_train, C=1.0, max_iter=1000, tol=1e-3):
    
    model = LinearSVC(C=C, max_iter=max_iter, tol=tol, dual=False)
    model.fit(x_train, y_train) 
    
    return model

In [31]:
result_dict['survived ~ linear_svc'] = build_model(linear_svc_fn,
                                                  'Survived',
                                                   FEATURES,
                                                   df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.8014059753954306
precision 0.7864077669902912
recall 0.7012987012987013
accuracy_count 456

Test data
accuracy 0.8531468531468531
precision 0.95
recall 0.6666666666666666
accuracy_count 122

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.8014059753954306
precision 0.7714285714285715
recall 0.7136563876651982
accuracy_count 456

Test data
accuracy 0.7762237762237763
precision 0.7735849056603774
recall 0.6721311475409836
accuracy_count 111

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.8
recall 0.721030042918455
accuracy_count 462

Test data
accuracy 0.7692307692307693
precision 0.7037037037037037
recall 0.6909090909090909
accuracy_count 110

Classification:  survived ~ sgd

Training data
accuracy 0.7047451669595782
precision 0.8421052631578947
recall 0.2909090909090909
accuracy_count 401

Test data
accuracy 0.65034965034965

In [32]:
def radius_neighbor_fn(x_train, y_train, radius=40.0):

    model = RadiusNeighborsClassifier(radius=radius)
    model.fit(x_train, y_train) 
    
    return model

In [33]:
result_dict['survived ~ radius_neighbors'] = build_model(radius_neighbor_fn,
                                                         'Survived',
                                                         FEATURES,
                                                         df)
compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.8014059753954306
precision 0.7864077669902912
recall 0.7012987012987013
accuracy_count 456

Test data
accuracy 0.8531468531468531
precision 0.95
recall 0.6666666666666666
accuracy_count 122

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.8014059753954306
precision 0.7714285714285715
recall 0.7136563876651982
accuracy_count 456

Test data
accuracy 0.7762237762237763
precision 0.7735849056603774
recall 0.6721311475409836
accuracy_count 111

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.8
recall 0.721030042918455
accuracy_count 462

Test data
accuracy 0.7692307692307693
precision 0.7037037037037037
recall 0.6909090909090909
accuracy_count 110

Classification:  survived ~ sgd

Training data
accuracy 0.7047451669595782
precision 0.8421052631578947
recall 0.2909090909090909
accuracy_count 401

Test data
accuracy 0.65034965034965

max_depth = None [ If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples ]

max_features = None [None -- max_features=n_features, auto -- then max_features=sqrt(n_features), sqrt -- then max_features=sqrt(n_features), log2 -- then max_features=log2(n_features)]

In [34]:
def decision_tree_fn(x_train, y_train, max_depth=None, max_features=None): 
    
    model = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
    model.fit(x_train, y_train)
    
    return model

In [36]:
result_dict['survived ~ decision_tree'] = build_model(decision_tree_fn,
                                                 'Survived',
                                                  FEATURES,
                                                  df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.8014059753954306
precision 0.7864077669902912
recall 0.7012987012987013
accuracy_count 456

Test data
accuracy 0.8531468531468531
precision 0.95
recall 0.6666666666666666
accuracy_count 122

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.8014059753954306
precision 0.7714285714285715
recall 0.7136563876651982
accuracy_count 456

Test data
accuracy 0.7762237762237763
precision 0.7735849056603774
recall 0.6721311475409836
accuracy_count 111

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.8
recall 0.721030042918455
accuracy_count 462

Test data
accuracy 0.7692307692307693
precision 0.7037037037037037
recall 0.6909090909090909
accuracy_count 110

Classification:  survived ~ sgd

Training data
accuracy 0.7047451669595782
precision 0.8421052631578947
recall 0.2909090909090909
accuracy_count 401

Test data
accuracy 0.65034965034965

In [37]:
def naive_bayes_fn(x_train,y_train, priors=None):
    
    model = GaussianNB(priors=priors)
    model.fit(x_train, y_train)
    
    return model

In [39]:
result_dict['survived ~ naive_bayes'] = build_model(naive_bayes_fn,
                                                    'Survived',
                                                    FEATURES,
                                                    df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.8014059753954306
precision 0.7864077669902912
recall 0.7012987012987013
accuracy_count 456

Test data
accuracy 0.8531468531468531
precision 0.95
recall 0.6666666666666666
accuracy_count 122

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.8014059753954306
precision 0.7714285714285715
recall 0.7136563876651982
accuracy_count 456

Test data
accuracy 0.7762237762237763
precision 0.7735849056603774
recall 0.6721311475409836
accuracy_count 111

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.8
recall 0.721030042918455
accuracy_count 462

Test data
accuracy 0.7692307692307693
precision 0.7037037037037037
recall 0.6909090909090909
accuracy_count 110

Classification:  survived ~ sgd

Training data
accuracy 0.7047451669595782
precision 0.8421052631578947
recall 0.2909090909090909
accuracy_count 401

Test data
accuracy 0.65034965034965